# Setup

In [2]:
import talib as ta
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pickle
import tiingo
import financedatabase as fd
import datetime
import financetoolkit
from utils.api_keys import API_KEYS

# Screening ETFs from Finance Database

We will use the [Finance Database](https://github.com/JerBouma/FinanceDatabase) package to identify an asset universe.

In [3]:
#Setting the api key as Financial Modeling Prep
API_KEY = API_KEYS['fmp']


In [4]:
# Initializing the ETFs database
etfs = fd.ETFs()

In [5]:
etfs.data

,name,currency,summary,category_group,category,family,exchange
symbol,,,,,,,
^ACWI,ISHARES TRUST,USD,The iShares MSCI ACWI ETF seeks to track the i...,Financials,Developed Markets,BlackRock Asset Management,NIM
^ADFI-IV,NFIELD DYNAMIC FIXED INCOME ETF,USD,The NFIELD DYNAMIC FIXED INCOME ETF (ADFI) is ...,Fixed Income,Corporate Bonds,NaN,ASE
^ADRE,INVESCO ACTIVELY M,USD,The Invesco Active REIT ETF is an actively man...,Real Estate,REITs,Invesco Investment Management,NIM
^ARB-EU,ALTSHARES MERGER ARBITRAGE ETF,USD,The ALTSHARES MERGER ARBITRAGE ETF seeks capit...,Alternatives,NaN,AltShares,ASE
^ARB-IV,ALTSHARES MERGER ARBITRAGE ETF,USD,The ALTSHARES MERGER ARBITRAGE ETF seeks capit...,Alternatives,NaN,AltShares,ASE
...,...,...,...,...,...,...,...
VGFPF,Vanguard Funds Public Limited Company - Vangua...,NaN,The Vanguard S&P 500 UCITS ETF (USD) Accumulat...,Equities,NaN,NaN,NaN
VFDEF,Vanguard Funds Public Limited Company - Vangua...,NaN,VFDEF is an exchange-traded fund (ETF) that ai...,Equities,NaN,NaN,NaN
WSDMF,WisdomTree Issuer ICAV - WisdomTree Europe Equ...,NaN,The WisdomTree Issuer ICAV - WisdomTree Europe...,Equities,NaN,NaN,NaN


In [8]:
etfs.data.loc['ARKK']


name                                             ARK Innovation ETF
currency                                                        USD
summary           The investment seeks long-term growth of capit...
category_group                               Information Technology
category                                                    Factors
family                                                ARK ETF Trust
exchange                                                        PCX
Name: ARKK, dtype: object

In [ ]:
#List columns options
etfs.show_options()

{'currency': array(['AUD', 'CAD', 'CHF', 'CNY', 'CZK', 'DKK', 'EUR', 'GBP', 'HUF',
        'ILA', 'ISK', 'JPY', 'KRW', 'MXN', 'NOK', 'QAR', 'RUB', 'SAR',
        'SEK', 'SGD', 'THB', 'TRY', 'TWD', 'USD'], dtype=object),
 'category_group': array(['Alternatives', 'Cash', 'Commodities', 'Communication Services',
        'Consumer Discretionary', 'Consumer Staples', 'Currencies',
        'Derivatives', 'Energy', 'Equities', 'Financials', 'Fixed Income',
        'Health Care', 'Industrials', 'Information Technology',
        'Materials', 'Real Estate', 'Utilities'], dtype=object),
 'category': array(['Alternative', 'Blend', 'Bonds', 'Cash', 'Commercial Real Estate',
        'Commodities Broad Basket', 'Communications',
        'Consumer Discretionary', 'Consumer Staples', 'Corporate Bonds',
        'Currencies', 'Derivatives', 'Developed Markets',
        'Emerging Markets', 'Energy', 'Equities', 'Factors', 'Financials',
        'Frontier Markets', 'Government Bonds', 'Growth', 'Health Care

In [44]:
# Filtering 'category_group' for Information Technology
etfs_data = etfs.select(category_group='Information Technology')
# Displaying the first 5 rows of the filtered data
etfs_data.head()

,name,currency,summary,category_group,category,family,exchange
symbol,,,,,,,
^ONEQ,FIDELITY COMMWLTH,USD,The Fidelity Nasdaq Composite ETF seeks to tra...,Information Technology,Large Cap,Fidelity Investments,NIM
^PNQI,INVESCO EXCHG TRAD,USD,The Invesco NASDAQ Internet ETF tracks the per...,Information Technology,NaN,Invesco Investment Management,NIM
^PSCT,INVESCO EXCH TRDII,USD,The Invesco S&P SmallCap Information Technolog...,Information Technology,Small Cap,Invesco Investment Management,NIM
00753L.TW,CTBC China 50 2X,TWD,The CTBC China 50 2X ETF aims to provide two t...,Information Technology,NaN,CTBC Securities Investment Trust,TAI
00757.TW,UPAMC NYSE FANG+ ETF,TWD,UPAMC NYSE FANG+ ETF aims to replicate the per...,Information Technology,NaN,Yuanta Securities Investment Trust,TAI


In [54]:
etfs_data.shape

(1297, 7)

In [56]:
etfs_data.isna().sum()

name                0
currency            4
summary            68
category_group      0
category          454
family            130
exchange            1
dtype: int64

In [ ]:
# grouping the data by 'category'
etfs_data.groupby('category', dropna=False).size().reset_index(name='count')

,category,count
0,Blend,3
1,Consumer Discretionary,1
2,Developed Markets,392
3,Emerging Markets,168
4,Factors,8
5,Frontier Markets,3
6,Growth,62
7,Large Cap,94
8,Mid Cap,9
9,Small Cap,35


Within the *"Information Technology"* will select only the two subcategories:
*   Developed Markets
*   Emerging Markets
We analyse where these ETFs are exchanged

In [70]:
# Filtering the data for 'Developed Markets' and 'Emerging Markets' by Exchange
top15_exchanges = etfs_data[etfs_data['category'].isin(['Developed Markets', 'Emerging Markets'])] \
    .groupby(['exchange'], dropna=False).size().reset_index(name='count').sort_values(by='count', ascending=False).reset_index(drop=True)
top15_exchanges

,exchange,count
0,FRA,70
1,BER,68
2,MUN,59
3,DUS,56
4,LSE,55
5,GER,40
6,EBS,35
7,PCX,27
8,HAM,27
9,MIL,27


There are 23 exchanges, we decide to keep only the 15 most significant ones

In [71]:
top15_exchanges_list = top15_exchanges['exchange'].head(15).tolist()
top15_exchanges_list

['FRA',
 'BER',
 'MUN',
 'DUS',
 'LSE',
 'GER',
 'EBS',
 'PCX',
 'HAM',
 'MIL',
 'MEX',
 'PAR',
 'TOR',
 'AMS',
 'HAN']

We can now make a new selection

In [81]:
# Obtaining Information Technology ETFs categorized as Emerging Markets or Developed markets from top 15 exchanges
tech_etfs = etfs.select(
    category_group='Information Technology',
    category=['Developed Markets', 'Emerging Markets'],
    exchange=top15_exchanges_list
)
# counting the number of ETFs by category and exchange in a pivot table
tech_etfs_pivot = tech_etfs_count.pivot(index='exchange', columns='category', values='count').fillna(0).astype(int)

# Adding totals for rows and columns
tech_etfs_pivot['Total'] = tech_etfs_pivot.sum(axis=1)
tech_etfs_pivot.loc['Total'] = tech_etfs_pivot.sum(axis=0)

tech_etfs_pivot

category,Developed Markets,Emerging Markets,Total
exchange,,,
AMS,11,2,13
BER,43,25,68
DUS,35,21,56
EBS,29,6,35
FRA,49,21,70
GER,33,7,40
HAM,17,10,27
HAN,3,5,8
LSE,39,16,55


We obtain a total of 534 ETFS traded in 15 Exchanges with approximately 70% categorized as developed markets and 30% as emerging markets
We can build our list of ETFs

In [ ]:
tech_etfs

,name,currency,summary,category_group,category,family,exchange
symbol,,,,,,,
018F.BE,ISHS INC-MSCI MALYSIA NEW,EUR,ISHS INC-MSCI MALYSIA NEW is an iShares ETF pr...,Information Technology,Emerging Markets,BlackRock Asset Management,BER
0253.BE,VANGUA.ESG INTL STOCK ETF,EUR,VANGUA.ESG INTL STOCK ETF is a Vanguard ETF fo...,Information Technology,Developed Markets,Vanguard Asset Management,BER
0254.BE,VANGUAR.ESG U.S.STOCK ETF,EUR,VANGUAR.ESG U.S.STOCK ETF is a Vanguard ETF th...,Information Technology,Developed Markets,Vanguard Asset Management,BER
025C.BE,VANG.EMERG.M.ST.IDX ETF,EUR,VANG.EMERG.M.ST.IDX ETF is a Vanguard ETF that...,Information Technology,Emerging Markets,Vanguard Asset Management,BER
025M.BE,VANG.TOT.INT.ST.IDX ETF,EUR,VANG.TOT.INT.ST.IDX ETF is a Vanguard ETF that...,Information Technology,Developed Markets,Vanguard Asset Management,BER
...,...,...,...,...,...,...,...
XWLD.L,Xtrackers MSCI World UCITS ETF 1C,GBP,The aim is for the investment to reflect the p...,Information Technology,Developed Markets,Xtrackers,LSE
ZPDT.BE,SPDR S+P US TECH.S.S.UETF,EUR,SPDR S+P US TECH.S.S.UETF is an exchange-trade...,Information Technology,Developed Markets,State Street Global Advisors,BER
ZPDT.DE,SPDR S&P U.S. Technology Select Sector UCITS ETF,EUR,The objective of the SPDR S&P U.S. Technology ...,Information Technology,Developed Markets,State Street Global Advisors,GER


In [87]:
tech_etfs_list = tech_etfs.index.tolist()
tech_etfs_list

['018F.BE',
 '0253.BE',
 '0254.BE',
 '025C.BE',
 '025M.BE',
 '2B79.DE',
 '2B79.F',
 'AH50.DE',
 'AH50.DU',
 'AH50.F',
 'AH50.L',
 'AH50.MU',
 'AH80.BE',
 'AH80.DU',
 'AH80.F',
 'AH80.HM',
 'AH80.MU',
 'AH81.BE',
 'AH81.DU',
 'AH81.F',
 'AH81.HM',
 'AH81.MU',
 'AH82.BE',
 'AH82.DU',
 'AH82.F',
 'AH82.HM',
 'AH82.MU',
 'AH83.BE',
 'AH83.MU',
 'AH87.MU',
 'AH89.BE',
 'AH89.DU',
 'AH89.F',
 'AH89.MU',
 'AH8H.DU',
 'AH8H.F',
 'AH8H.HM',
 'AH8H.MU',
 'AH8J.BE',
 'AH8J.DU',
 'AH8J.F',
 'AH8J.MU',
 'AH8P.HM',
 'AH8R.DU',
 'AH8R.F',
 'AH8U.MU',
 'AH8V.BE',
 'AH8V.DU',
 'AH8V.F',
 'AH8V.HM',
 'AH8V.MU',
 'BUNH.DE',
 'BUNH.DU',
 'BUNH.F',
 'BUZZ',
 'CBUSA-USD.SW',
 'CBUSA.SW',
 'CC1.MI',
 'CC1.PA',
 'CC1E.SW',
 'CC1G.L',
 'CC1U.L',
 'CC1U.PA',
 'CC1USD.SW',
 'CCAU.L',
 'CCAUN.MX',
 'CCEI.TO',
 'CD47.BE',
 'CD47.DE',
 'CD47.DU',
 'CD47.F',
 'CD47.HM',
 'CD47.MU',
 'CD5.PA',
 'CD5E.SW',
 'CD8.PA',
 'CD9.PA',
 'CE2D.L',
 'CE8.PA',
 'CE8E.SW',
 'CE8G.DE',
 'CFPM.F',
 'CHIK',
 'CHIK.MX',
 'CQQQ',
 'DG

We need to remove duplicate ETFs which are traded in multiple exchanges, like 
 *   'AH81.BE',
 *  'AH81.DU',
 *  'AH81.F',
 *  'AH81.HM',
 *  'AH81.MU',